In [ ]:

!pip install sentence-transformers
!pip install transformers
!pip install datasets
!pip install torch


In [ ]:
import pandas as pd
import numpy as np
import torch
import time
from datasets import load_dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from sentence_transformers import SentenceTransformer, util

In [ ]:

!pip install kaggle


from google.colab import files
files.upload()


!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/


!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d thoughtvector/customer-support-on-twitter


!unzip customer-support-on-twitter.zip


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/thoughtvector/customer-support-on-twitter
License(s): CC-BY-NC-SA-4.0
 99% 167M/169M [00:13<00:00, 15.6MB/s]
100% 169M/169M [00:13<00:00, 13.3MB/s]
Archive:  customer-support-on-twitter.zip
  inflating: sample.csv              
  inflating: twcs/twcs.csv           


In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('/content/twcs/twcs.csv')


print(df.head())




   tweet_id   author_id  inbound                      created_at  \
0         1  sprintcare    False  Tue Oct 31 22:10:47 +0000 2017   
1         2      115712     True  Tue Oct 31 22:11:45 +0000 2017   
2         3      115712     True  Tue Oct 31 22:08:27 +0000 2017   
3         4  sprintcare    False  Tue Oct 31 21:54:49 +0000 2017   
4         5      115712     True  Tue Oct 31 21:49:35 +0000 2017   

                                                text response_tweet_id  \
0  @115712 I understand. I would like to assist y...                 2   
1      @sprintcare and how do you propose we do that               NaN   
2  @sprintcare I have sent several private messag...                 1   
3  @115712 Please send us a Private Message so th...                 3   
4                                 @sprintcare I did.                 4   

   in_response_to_tweet_id  
0                      3.0  
1                      1.0  
2                      4.0  
3                      5.0  
4

In [ ]:
print(f"Length of queries: {len(queries)}")
print(f"Length of responses: {len(responses)}")


Length of queries: 1537843
Length of responses: 1273931


In [ ]:
# Filter only relevant inbound (customer) and outbound (support) messages
inbound = df[df['inbound'] == True]
outbound = df[df['inbound'] == False]

# Merge inbound and outbound messages based on tweet IDs
merged = pd.merge(inbound, outbound, left_on='tweet_id', right_on='in_response_to_tweet_id', suffixes=('_query', '_response'))

# Extract the relevant columns
queries = merged['text_query'].tolist()
responses = merged['text_response'].tolist()

# Ensure the lists are of the same length
print(f"Length of queries: {len(queries)}")
print(f"Length of responses: {len(responses)}")

fine_tuning_data = pd.DataFrame({
    'queries': queries,
    'responses': responses
})

fine_tuning_data.to_csv("customer_support_fine_tuning.csv", index=False)


Length of queries: 1261888
Length of responses: 1261888


In [ ]:
min_length = min(len(queries), len(responses))

# Truncate both lists to the same length
queries = queries[:min_length]
responses = responses[:min_length]




In [ ]:
# Load the dataset
dataset = load_dataset('csv', data_files='customer_support_dataset.csv')

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


In [ ]:

tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

def tokenize_function(examples):
    tokens = tokenizer(examples['queries'], truncation=True, padding='max_length', max_length=128)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [ ]:

# Load and configure the model
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.config.pad_token_id = model.config.eos_token_id

In [ ]:

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=200,
    report_to="tensorboard"  # Enable TensorBoard for visualizing training loss
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
)

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

def tokenize_function(examples):
    # Encode the queries and set the labels as the input_ids
    tokens = tokenizer(examples['queries'], truncation=True, padding='max_length', max_length=128)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_dataset = dataset.map(tokenize_function, batched=True)



Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [ ]:
# Measure training time
start_time = time.time()

trainer.train()

end_time = time.time()
training_time = end_time - start_time
print(f"Training Time: {training_time} seconds")

Step,Training Loss


Training Time: 24.598041772842407 seconds


In [ ]:
# Save the fine-tuned model
model.save_pretrained("./fine_tuned_gpt2")
tokenizer.save_pretrained("./fine_tuned_gpt2")

('./fine_tuned_gpt2/tokenizer_config.json',
 './fine_tuned_gpt2/special_tokens_map.json',
 './fine_tuned_gpt2/vocab.json',
 './fine_tuned_gpt2/merges.txt',
 './fine_tuned_gpt2/added_tokens.json')

In [ ]:

# Load the fine-tuned model and tokenizer
fine_tuned_model = GPT2LMHeadModel.from_pretrained("./fine_tuned_gpt2")
fine_tuned_tokenizer = GPT2Tokenizer.from_pretrained("./fine_tuned_gpt2")

In [ ]:
# Generate a response
def generate_response(prompt, max_length=150):
    inputs = fine_tuned_tokenizer.encode(prompt, return_tensors="pt")
    outputs = fine_tuned_model.generate(inputs, max_length=max_length, num_return_sequences=1, pad_token_id=fine_tuned_tokenizer.eos_token_id)
    return fine_tuned_tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:

# Load SentenceTransformer model for embeddings
embedder = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# Define a evaluation function using cosine similarity
def evaluate_model_cosine_similarity(model, tokenizer, test_queries, expected_responses):
    generated_responses = [generate_response(query) for query in test_queries]

    # Compute embeddings for generated and expected responses
    generated_embeddings = embedder.encode(generated_responses, convert_to_tensor=True)
    expected_embeddings = embedder.encode(expected_responses, convert_to_tensor=True)

    # Calculate cosine similarities
    similarities = []
    for gen_emb in generated_embeddings:
        sim_scores = util.pytorch_cos_sim(gen_emb, expected_embeddings)
        similarities.append(float(torch.max(sim_scores).cpu().numpy()))

    # Calculate average similarity as a proxy for accuracy
    average_similarity = np.mean(similarities)
    return average_similarity

In [ ]:

# Define a set of test queries and expected responses
test_queries = [
    "How do I reset my password?",
    "What is the status of my order?",
    "Can I change my shipping address?",
    "How do I return an item?",
    "My payment was declined, what should I do?",
    "How can I track my shipment?",
    "I received a damaged product, how do I get a replacement?",
    "Can I cancel my order?",
    "How do I update my email address?",
    "What is your refund policy?",
    "How can I contact customer support?",
    "Do you offer expedited shipping?",
    "I forgot my account password, how do I recover it?",
    "Where can I find my order history?",
    "What payment methods do you accept?",
    "How do I report a fraudulent transaction?",
    "Can I change my billing address?",
    "How long does it take to process a refund?",
    "I didn't receive my order, what should I do?",
    "How do I use a discount code?",
    "How do I apply for a refund?",
    "Where is my package?",
    "Can I get an exchange for my item?",
    "What are your business hours?",
    "How can I update my payment information?",
    "Can I change my delivery time?",
    "Do you offer gift wrapping?",
    "How do I check the status of my refund?",
    "How can I verify my account?",
    "Can I return an item without the receipt?",
    "What should I do if I receive the wrong item?",
    "How can I find my order confirmation?",
    "Do you offer international shipping?",
    "How do I report an issue with my account?",
    "Can I add items to an existing order?",
    "How do I cancel a subscription?",
    "Where can I find your privacy policy?",
    "Can I request a replacement for a defective item?",
    "How do I change my shipping method?",
    "Can I use multiple discount codes on one order?",
    "How do I opt out of promotional emails?",
    "What is your policy on lost shipments?",
    "How do I update my shipping address for a current order?",
    "Can I return a gift?",
    "How do I track a return shipment?",
    "What is the process for a warranty claim?",
    "How can I contact the sales team?",
    "Can I get a price adjustment for a sale item?",
    "How do I sign up for your newsletter?",
    "Can I change my order after it's been placed?",
    "How do I request a product review?",
    "Do you offer product exchanges?",
    "What are your delivery options?",
    "How do I get help with a technical issue?",
    "Can I add a special request to my order?",
    "What is your policy on restocking fees?",
    "How can I update my contact information?",
    "What should I do if my order is incomplete?",
    "Can I check the status of my return?",
    "How do I request a refund for a digital product?",
    "Can I track my order using an order number?",
    "How do I resolve issues with a service subscription?",
    "What should I do if my package is delayed?",
    "How can I request a call back from customer service?",
    "Can I get a discount for bulk purchases?",
    "How do I report a technical issue with your website?",
    "What is your policy on custom orders?",
    "How do I update my delivery instructions?",
    "Can I get a replacement for a missing item?",
    "How do I return an item purchased with a gift card?",
    "What is the process for disputing a charge?",
    "Can I get a refund for a service I no longer need?",
    "How do I track a package that's been shipped?",
    "What are the steps for a product recall?",
    "Can I use a discount code after placing an order?",
    "How do I update my payment method for a subscription?",
    "What should I do if I haven't received a confirmation email?",
    "How can I cancel an appointment?",
    "Can I get a refund for an item I purchased from a third party?",
    "What is your policy on changing order details?",
    "How do I get a copy of my invoice?",
    "Can I request an exchange for a product that was purchased online?",
    "What is the status of my warranty claim?",
    "How do I change my delivery address after an order has been placed?",
    "Can I get a refund if my order was delayed?",
    "How do I opt out of automatic renewals for a subscription?",
    "What should I do if I receive a defective product?",
    "Can I get a refund for a digital service?",
    "How do I check the balance on my gift card?",
    "Can I update my shipping address for future orders?",
    "What are the options for expedited delivery?",
    "How do I return an item purchased during a sale?",
    "Can I get a replacement for an item that was damaged during shipping?",
    "How do I submit a complaint about a service?",
    "Can I use a gift card and a discount code together?",
    "How do I update my preferences for promotional emails?",
    "What is the process for returning an item without a receipt?",
    "How can I get assistance with my account?",
    "Can I track a package if I lost my tracking number?",
    "What should I do if my product is not as described?",
    "How do I request a product swap?",
    "Can I return an item after the return window has closed?",
    "What is your policy on returning items bought in a bundle?",
    "How do I get a refund for a subscription service I didn't use?",
    "Can I request a discount on a large order?",
    "How do I resolve an issue with a digital purchase?",
    "What is your policy on exchanges for personalized items?",
    "How do I change my shipping method after an order is placed?",
    "Can I get a refund for a service that was canceled?",
    "How do I contact customer support for an issue with a product?",
    "What are the options for return shipping?",
    "How can I update my address for delivery?",
    "What should I do if I accidentally ordered the wrong item?",
    "Can I get a refund for a service that did not meet my expectations?",
    "How do I cancel an order that has already been shipped?",
    "Can I get a replacement if my product is not working properly?",
    "How do I check the status of a return request?",
    "What is the process for getting a refund for a damaged item?",
    "How do I handle issues with a service subscription renewal?",
    "Can I get a refund for an item purchased with a discount code?",
    "How do I request a return for an item purchased during a promotion?",
    "What should I do if I haven’t received my refund yet?",
    "How do I get a replacement for a product that arrived defective?",
    "Can I use a promotional code on top of a discount?",
    "What is your policy on returns for opened items?",
    "How do I check the status of my subscription?",
    "Can I change the delivery address for a shipment already in transit?",
    "What should I do if I receive a different item than ordered?",
    "How do I update my payment information for future orders?",
    "Can I request an extension for a return period?",
    "How do I get a replacement for an item that broke after use?",
    "Can I return an item purchased from a third-party seller?",
    "How do I handle issues with a subscription service?",
    "Can I get a refund if my item arrives late?",
    "How do I submit a request for a product review?",
    "Can I get assistance with a billing issue?",
    "What is the process for returning a gift received through online shopping?"
]

expected_responses = [
    "To reset your password, go to the settings page and click on 'Forgot Password'.",
    "Your order is being processed and will be shipped soon.",
    "Yes, you can change your shipping address by going to your account settings.",
    "To return an item, please visit our returns page and follow the instructions.",
    "If your payment was declined, please check your card details or contact your bank for assistance.",
    "You can track your shipment using the tracking number provided in your confirmation email.",
    "If you received a damaged product, please contact our support team for a replacement.",
    "To cancel your order, please visit your order history page and select the 'Cancel Order' option.",
    "To update your email address, go to your account settings and change your email.",
    "Our refund policy details are available on our website under the 'Refunds' section.",
    "You can contact customer support via email, phone, or live chat on our website.",
    "Yes, we offer expedited shipping options at checkout.",
    "To recover your account password, go to the login page and click on 'Forgot Password'.",
    "You can view your order history by logging into your account and visiting the 'Orders' section.",
    "We accept various payment methods including credit/debit cards and PayPal.",
    "To report a fraudulent transaction, please contact our support team immediately.",
    "To change your billing address, go to your account settings and update your address.",
    "Refund processing times vary, but typically take 5-7 business days.",
    "If you didn't receive your order, please contact our support team to investigate the issue.",
    "To use a discount code, enter it at checkout on the payment page.",
    "To apply for a refund, visit our refund request page and follow the steps outlined there.",
    "You can check the status of your package using the tracking number provided.",
    "Yes, you can get an exchange by contacting our support team with your order details.",
    "Our business hours are from 9 AM to 6 PM, Monday through Friday.",
    "To update your payment information, go to your account settings and edit your payment details.",
    "Changing your delivery time can be done through our support team or account settings.",
    "We offer gift wrapping options at checkout. You can select it during your purchase.",
    "To check the status of your refund, log into your account and view your refund status on the order page.",
    "Verify your account by following the verification email sent to your registered email address.",
    "You can return an item without the receipt by providing the order number and account details.",
    "If you receive the wrong item, please contact our support team to arrange for a replacement.",
    "Your order confirmation can be found in the confirmation email we sent you or in your account order history.",
    "Yes, we offer international shipping to select countries.",
    "Report any account issues by contacting our support team with details of the problem.",
    "To add items to an existing order, please contact our support team before the order is processed.",
    "To cancel a subscription, go to your account settings and follow the cancellation process.",
    "Our privacy policy is available on our website under the 'Privacy Policy' section.",
    "If you need a replacement for a defective item, contact our support team with your order details.",
    "To change your shipping method, contact our support team before the order is shipped.",
    "Multiple discount codes cannot be used on a single order. Only one code per order is allowed.",
    "To opt out of promotional emails, update your preferences in your account settings.",
    "Lost shipments can be reported to our support team for further assistance.",
    "Update your shipping address by contacting our support team or through your account settings.",
    "You can return a gift by contacting our support team with the gift details and order number.",
    "Track your return shipment using the return tracking number provided in the confirmation email.",
    "For warranty claims, follow the process outlined on our warranty page and contact support if needed.",
    "Contact the sales team through our website's contact form or by phone.",
    "Price adjustments for sale items can be requested by contacting our support team.",
    "Sign up for our newsletter by entering your email address on our subscription page.",
    "Changes to orders after they are placed may not be possible. Contact support for assistance.",
    "Submit a product review by visiting the product page and using the review submission form.",
    "Yes, product exchanges are possible. Contact our support team to arrange an exchange.",
    "Delivery options are listed at checkout. Choose the option that best suits your needs.",
    "For technical issues, contact our support team or visit our help center for troubleshooting tips.",
    "Special requests can be added to your order by contacting our support team before the order is processed.",
    "Restocking fees are applicable for certain items. Refer to our returns policy for details.",
    "Update your contact information by logging into your account and editing your contact details.",
    "If your order is incomplete, please contact our support team to resolve the issue.",
    "Check the status of your return by logging into your account and viewing the return details.",
    "For digital product refunds, follow the refund request process outlined on our website.",
    "Track your order using the order number provided in your confirmation email.",
    "Resolve issues with service subscriptions by contacting our support team for assistance.",
    "If your package is delayed, contact our support team to check the status of your shipment.",
    "Request a call back from customer service by filling out the call back request form on our website.",
    "Bulk purchase discounts can be discussed with our sales team or through our support team.",
    "Report technical issues with our website by contacting our support team or using the feedback form.",
    "Custom orders follow a different process. Contact our support team for details on how to place a custom order.",
    "Update your delivery instructions by contacting our support team or through your account settings.",
    "If an item is missing, contact our support team to arrange for a replacement.",
    "Return items purchased with a gift card by contacting our support team with the order details.",
    "Disputing a charge involves contacting our support team and providing the necessary details.",
    "Refunds for unused services can be requested by contacting our support team with your order information.",
    "Track a shipped package using the tracking number provided in your shipment confirmation email.",
    "Product recall information will be communicated through our website and email notifications.",
    "Discount codes cannot be used retroactively. Apply codes during checkout for new orders.",
    "Update payment methods for subscriptions through your account settings or by contacting our support team.",
    "If you haven't received a confirmation email, check your spam folder or contact our support team.",
    "Cancel an appointment by contacting the scheduling department or using our online scheduling tool.",
    "Refunds for third-party purchases can be requested through the original seller or platform.",
    "Changing order details post-purchase may not be possible. Contact our support team for assistance.",
    "Obtain a copy of your invoice by logging into your account and downloading the invoice from your order history.",
    "Requests for exchanges of online purchases should be directed to our support team with your order details.",
    "Check the status of a warranty claim by contacting our support team or through your account settings.",
    "Change delivery addresses by contacting our support team as soon as possible before the order is shipped.",
    "Refunds for delayed orders can be requested through our support team. Provide order details for assistance.",
    "Opt out of automatic renewals by adjusting your subscription settings in your account.",
    "If you receive a defective product, contact our support team for a replacement or refund.",
    "Refunds for digital services can be requested through our support team with your purchase details.",
    "Check the balance on your gift card by visiting our gift card balance page on our website.",
    "Update your shipping address for future orders by editing your address in your account settings.",
    "Expedited delivery options are available at checkout. Choose the option that suits your needs.",
    "Return items purchased during a sale by following our standard return process outlined on our website.",
    "Replacement requests for damaged items during shipping should be directed to our support team.",
    "Submit a complaint about a service by contacting our support team with details of the issue.",
    "Use gift cards and discount codes separately, as they cannot be combined for a single order.",
    "Update promotional email preferences in your account settings or by following the unsubscribe link.",
    "Return items without a receipt by providing order details and contacting our support team.",
    "For account assistance, contact our support team or visit our help center for guidance.",
    "Lost tracking numbers can be retrieved by contacting our support team with your order details.",
    "If the product is not as described, contact our support team for a resolution or replacement.",
    "Request a product swap by contacting our support team with the details of the exchange.",
    "Returning items after the return window requires contacting our support team for approval.",
    "Returning bundle items follows the same process as individual returns. Contact support for details.",
    "Refunds for unused subscription services can be requested through our support team with your account information.",
    "Discuss bulk purchase discounts with our sales team or support team for applicable offers.",
    "Resolve digital purchase issues by contacting our support team with details of the problem.",
    "Custom item exchanges are subject to our exchange policy. Contact support for more information.",
    "Change shipping methods by contacting our support team before the order is dispatched.",
    "Refunds for canceled services can be processed by contacting our support team with your details.",
    "For issues with a product, contact our customer support team for assistance and resolution.",
    "Return shipping options are detailed on our returns page. Choose the method that works best for you.",
    "Update your delivery address by contacting our support team or through your account settings.",
    "If you ordered the wrong item, contact our support team for assistance with returns or exchanges.",
    "Refunds for services not meeting expectations can be processed by contacting our support team.",
    "Canceling a shipped order may not be possible. Contact support to explore available options.",
    "Get a replacement for a malfunctioning product by contacting our support team with your order details.",
    "Check return request status by logging into your account and viewing the return details.",
    "Follow the outlined process for damaged item refunds by contacting our support team for assistance.",
    "Service subscription issues can be resolved by contacting our support team with your subscription details.",
    "Refunds for discount code purchases can be requested by contacting our support team with your order information.",
    "Return items bought during a promotion by following the standard return process and contacting support.",
    "If a refund is delayed, contact our support team for an update on your refund status.",
    "Replacement requests for defective items should be made by contacting our support team promptly.",
    "Promotional codes should be applied at checkout. They cannot be combined with other discounts.",
    "Opened items can be returned based on our return policy. Refer to our website for details.",
    "Check subscription status by logging into your account and viewing your subscription details.",
    "Contact support to update the delivery address for an in-transit shipment if possible.",
    "Report issues with received items by contacting our support team for resolution.",
    "Update payment information for future orders by accessing your account settings.",
    "Request an extension for a return period by contacting our support team with your order details.",
    "Get a replacement for a product that broke after use by contacting our support team.",
    "Return items purchased from third-party sellers by contacting the seller directly for the return process.",
    "Address subscription service issues by contacting our support team for resolution.",
    "Request refunds for late items by contacting our support team with your order details.",
    "Submit a request for a product review by visiting the product page on our website.",
    "Resolve billing issues by contacting our support team with the details of the problem.",
    "Return gifts purchased through online shopping by following the return process outlined on our website."
]


In [ ]:

# Evaluate model
average_similarity = evaluate_model_cosine_similarity(fine_tuned_model, fine_tuned_tokenizer, test_queries, expected_responses)
print(f"Average Cosine Similarity: {average_similarity:.2f}")

Average Cosine Similarity: 0.79


In [ ]:
import time

# Function to evaluate and measure time
def evaluate_and_measure_time(model, tokenizer, test_queries, expected_responses, description=""):
    start_time = time.time()
    average_similarity = evaluate_model_cosine_similarity(model, tokenizer, test_queries, expected_responses)
    end_time = time.time()
    evaluation_time = end_time - start_time

    print(f"{description} - Average Cosine Similarity: {average_similarity:.2f}")
    print(f"{description} - Evaluation Time: {evaluation_time:.2f} seconds")

    return average_similarity, evaluation_time

# Evaluate the model before fine-tuning
print("Evaluating before fine-tuning...")
average_similarity_before, evaluation_time_before = evaluate_and_measure_time(model, tokenizer, test_queries, expected_responses, description="Before Fine-Tuning")

# Fine-tuning the model
start_time_train = time.time()
trainer.train()
end_time_train = time.time()
training_time = end_time_train - start_time_train
print(f"Training Time: {training_time:.2f} seconds")

model.save_pretrained("./fine_tuned_gpt2")
tokenizer.save_pretrained("./fine_tuned_gpt2")


fine_tuned_model = GPT2LMHeadModel.from_pretrained("./fine_tuned_gpt2")
fine_tuned_tokenizer = GPT2Tokenizer.from_pretrained("./fine_tuned_gpt2")

# Evaluate model after fine-tuning
print("Evaluating after fine-tuning...")
average_similarity_after, evaluation_time_after = evaluate_and_measure_time(fine_tuned_model, fine_tuned_tokenizer, test_queries, expected_responses, description="After Fine-Tuning")

# Calculate improvement
similarity_improvement = average_similarity_after - average_similarity_before
time_difference = evaluation_time_after - evaluation_time_before

print(f"Improvement in Average Cosine Similarity: {similarity_improvement:.2f}")
print(f"Difference in Evaluation Time: {time_difference:.2f} seconds")


Evaluating before fine-tuning...
Before Fine-Tuning - Average Cosine Similarity: 0.79
Before Fine-Tuning - Evaluation Time: 26.63 seconds


Step,Training Loss


Training Time: 34.41 seconds
Evaluating after fine-tuning...
After Fine-Tuning - Average Cosine Similarity: 0.79
After Fine-Tuning - Evaluation Time: 22.43 seconds
Improvement in Average Cosine Similarity: 0.00
Difference in Evaluation Time: -4.19 seconds
